PDA 2019 Kaggle Data Competition
For the Data competition the task was to build a recommender system using information from a movie rating database. The training data was composed of user's ratings of different movies and information on the movies (genre, visual features, release year, tags and title). The objective of the system was to recommend to each user in the test set their top 10 favorite movies. 
The algorithm I decided to use for this assignment was SVD. SVD stands for Singular Value Decomposition which works using matrix factorization. Matrix factorization breaks down one matrix into the product of multiple matrices. Although there are many ways to do matrix factorization, SVD is quite powerful when used for recommendations. It decomposes a matrix into two unitary matrices and a diagonal matrix.

                                               R=UΣV^T
                                               
Where R is our user ratings matrix, U is the user features matrix, Σ is the diagonal matrix of singular values and V^T is the movie features matrix. U and V^T are orthogonal, U represents how much users ‘like’ each feature and V^T represents how relevant each feature is to a movie. To get the lower rank approximation, I use the matrices and keep only the top k features, which are the k most relevant taste and preference vectors.
My objective with this model is to return the movies with the highest predicted rating that a specific user has not yet seen. The reason I have chosen the svds function from scipy is because it allowed me to choose the number of latent factors, and also when using the SVD function from surprise, the result I got back from the algorithms in the competition was half that of the SVDS model, also there is no need to truncate after calculation. 

# Loading the data
I first began by loading all the packages I used and loading all the datasets that were offered in the competition.

In [67]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
rating_df = pd.read_csv('../data/train-PDA2019.csv',sep=',')
rating_df.head()
rating_df.head()
movies_df = pd.read_csv('../data/content-PDA2019.csv',sep=',')
rating_df.head()
test = pd.read_csv('../data/test-PDA2019.csv')


### Transforming the original dataset
I pivot the original ratings dataset as to have every usedID on 1 row and every movieID on 1 column. Then we can compute the average rating for each user, then normalize the data by using the average of each user in R_demeaned.

In [68]:
r_df = rating_df.pivot(index = 'userID', columns ='itemID', values = 'rating')
r_df

itemID,89,93,94,95,97,98,100,101,102,104,...,3929,3930,3931,3932,3937,3938,3945,3946,3950,3952
userID,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Calculating the mean for each user and demeaning the original data
Then we can compute the average rating for each user, then normalize the data by using the average of each user in R_demeaned. We used the pivoted dataset to replace the values for each movies with the average for each user. Doing this we normalize and set the unknown rates with the user mean(0 after substraction). To be able to calculate our 3 matrixes, the data has to be demeaned.

In [69]:
users_mean=np.array(r_df.mean(axis=1))
users_mean

array([4.        , 4.07692308, 4.2972973 , ..., 3.52173913, 4.01190476,
       3.55744681])

In [70]:
R_demeaned=r_df.sub(r_df.mean(axis=1), axis=0)
R_demeaned=R_demeaned.fillna(0).as_matrix()
R_demeaned

c:\users\abuliga\pycharmprojects\thisnewproject\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [84]:
R_demeaned.shape

(5690, 1824)

### Using SVDS to calculate the U,Sigma and V^T matrices
Now my matrix is properly formatted and normalized, I can go on with the singular value decomposition. I used the svds function from scipy because it lets me choose how many latent factors I use to approimate the original ratings matrix (instead of having to truncate afterwards). I defined the 3 matrixes above, which will be used to calculate our reconstructed matrix below.
For movies, predictions from lower rank matrices with values of k between roughly 20 and 100 have been found to be the best at generalizing to unseen data.

In [72]:
U, sigma, Vt = svds(R_demeaned, k = 20,maxiter=20)

The svds function just prints out the values of the diagonal matrix we defined above as Σ, but for matrix multiplication we need to convert it into a diagonal matrix.

In [73]:
sigma = np.diag(sigma)
sigma

array([[ 42.49678016,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,  42.83402439,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,  43.6471488 ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,  

Now we have everything we need to predict the ratings for all users and all movies in the original dataset. The way the predictions are computed is by using dot product(matrix) and multiplying the U,Σ and then use that dot product to multiplicate again with  the Vt matrix to get the reconstructed matrix. The reconstructed matrix with the the mean of the users gives me back the predicted ratings for each user and for each movie.

### Computing the user predictions
Below I obtained all the predicted values for each user that need to be combined with the mean rating of each user. Basically this is the reconstructed A matrix.

In [ ]:
predicted_ratings = np.dot(np.dot(U, sigma), Vt)
predicted_ratings

Now I can add back the user`s means.

In [ ]:
all_user_predicted_ratings = predicted_ratings+users_mean.reshape(-1, 1)

Now that we have all the predicted ratings,I used the columns from the pivoted dataset to match them to the corresponding userID and movieID. 

In [ ]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = r_df.columns)
preds_df.index=r_df.index.values
preds_df

### Finding the top 10 recommendations for each user
In the preds_df dataframe, each column of itemID refers to a movie and each row represents on user. What I did next is sort the ratings in descendig order to get the top 10 predited ratings for each user. Also, I used the userID column from thetest dataset as the index to extract the predictions only for the users in the test set.

In [80]:
users = test.loc[:,'userID']
users

0           1
1           3
2          11
3          29
4          31
        ...  
1987    12047
1988    12051
1989    12061
1990    12063
1991    12073
Name: userID, Length: 1992, dtype: int64

Finally, I am going to build the recommendations dataset with the following for loop: I used the index above to determine the users that I want to predict for, then I used the top10 list to get the top 10 reccomended movies for each user after sorting them in descending order. Then I transforme the user predictions into a string and concatenated them together in a series where the key value is the userID and the values are the top 10 recommended unseen movies for each user.

In [77]:
col_names = preds_df.columns
ind = 0
for j in range(len(users)):
    user = users[j]
    user_pred = []
    rating = preds_df.loc[user,:]
    rating = rating.sort_values(ascending=False)
    top10 = rating[0:10]
    indices = top10.index[:].tolist()
    indices_str = str(indices).strip('[]').replace(",", " ")
    user_pred = [indices]
    test.loc[ind] = pd.Series({'userID':user,'recommended_itemIDs':indices_str})
    ind += 1

Here I have the final output where I obtained the top 10 movies for each user and the movies` respective IDs

In [ ]:
test

In the end I just export my final Series to a csv file as to upload the result to the Kaggle competition page.

In [79]:
test.to_csv(path_or_buf = 'recommendations.csv', 
                  index = False,
                  header = True, sep = ',')